In [1]:
!pip install openai python-dotenv


[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import dotenv
import openai
import re
dotenv.load_dotenv('../.env')

True

In [56]:
hero_name = "Léa"
story_idea = "une jeune sorcière en formation qui doit prouver ses compétences en sauvant son village d'un sortilège maléfique"
instructions = """
Consignes:
Un chapitre doit être court et se terminer par un cliffhanger.
Un Chapitre est divisé en plusieurs court paragraphes.
Un paragraphes contient Deux phrases au maximum."""
imagen_instructions = """-----
Write a very short one sentence visual description of the scene. Include a very short description of the character and its environnement. Keep it short and simple."""

In [12]:
def generate_prompt(story_idea, hero_name, instructions):
    prompt = f"""Ecris une histoire pour enfant composée de 5 chapitres.
{story_idea}
Personnage principal: {hero_name}
{instructions}
"""
    return prompt

In [13]:
def generate_story(story_idea, hero_name, instructions):
    prompt = generate_prompt(story_idea, hero_name, instructions)
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role":"user", "content": prompt}]
    )
    full_text = response.choices[0].message.content
    return full_text

In [14]:
story = generate_story(story_idea, hero_name, instructions)
print(story)

Chapitre 1: Les débuts de Léa

Léa, une jeune sorcière en formation, vivait dans un petit village paisible entouré de forêts enchantées. Depuis toute petite, elle rêvait de devenir une grande sorcière et d'aider les habitants de son village avec ses pouvoirs magiques. Un jour, son plus grand rêve se réalisa lorsqu'elle fut acceptée à l'Académie des Sorcières. Elle quitta son village avec enthousiasme, les yeux remplis d'étoiles. Mais à peine arrivée, elle fut confrontée à un défi inattendu : un sortilège maléfique enveloppa son village natal, plongeant ses habitants dans le désespoir. Léa savait qu'elle devait revenir et sauver son village coûte que coûte.

Chapitre 2: La quête de Léa

Déterminée à sauver son village, Léa commença sa quête. Elle se rendit dans la bibliothèque de l'Académie où elle étudia les sorts et les enchantements pour contrer le sortilège maléfique. Elle découvrit qu'elle devait réunir sept ingrédients magiques dispersés dans le royaume enchanté, afin de créer un 

In [33]:

def format_raw_story(story):

    # split story into chapters
    pattern = r"(Chapitre \d+: [^\n]+)\n\n(.*?)(?=\n\nChapitre|$)"

    # Find all matches
    story_by_chapters = re.findall(pattern, story, re.DOTALL)

    # Split the chapters into paragraphs of at least 100 characters
    story_by_chapters_and_paragraphs = []
    for chapter in story_by_chapters:
        chapter_title = chapter[0]
        chapter_content = chapter[1]
        chapter_content_splited = []
        for sentence in chapter_content.split("."):
            sentence = sentence.strip()
            if sentence == "": continue
            if len(chapter_content_splited) == 0:
                chapter_content_splited.append(sentence)
            elif len(chapter_content_splited[-1]) < 100:
                chapter_content_splited[-1] += f". {sentence}"
            else:
                chapter_content_splited.append(sentence)
        story_by_chapters_and_paragraphs.append((chapter_title, chapter_content_splited))
    
    return story_by_chapters_and_paragraphs

In [34]:
formated_story = format_raw_story(story)

In [60]:
formated_story

[('Chapitre 1: Les débuts de Léa',
  ['Léa, une jeune sorcière en formation, vivait dans un petit village paisible entouré de forêts enchantées',
   "Depuis toute petite, elle rêvait de devenir une grande sorcière et d'aider les habitants de son village avec ses pouvoirs magiques",
   "Un jour, son plus grand rêve se réalisa lorsqu'elle fut acceptée à l'Académie des Sorcières. Elle quitta son village avec enthousiasme, les yeux remplis d'étoiles",
   'Mais à peine arrivée, elle fut confrontée à un défi inattendu : un sortilège maléfique enveloppa son village natal, plongeant ses habitants dans le désespoir',
   "Léa savait qu'elle devait revenir et sauver son village coûte que coûte"]),
 ('Chapitre 2: La quête de Léa',
  ["Déterminée à sauver son village, Léa commença sa quête. Elle se rendit dans la bibliothèque de l'Académie où elle étudia les sorts et les enchantements pour contrer le sortilège maléfique",
   "Elle découvrit qu'elle devait réunir sept ingrédients magiques dispersés 

In [61]:
def text_to_images_desc(chapter_content):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role":"user", "content": f"""{chapter_content}
{imagen_instructions}"""}
                  ],
        max_tokens=50
    )
    full_text = response.choices[0].message.content
    return full_text

                   
    

In [62]:
story_with_images_desc = dict()
for i, chapter in enumerate(formated_story):
    chapter_title = chapter[0]
    chapter_content = chapter[1]
    story_with_images_desc[f'chapter_{i}'] = dict()
    story_with_images_desc[f'chapter_{i}']['title'] = chapter_title
    print(chapter_title)
    for j, paragraph in enumerate(chapter_content):
        paragraph_image_desc = text_to_images_desc(paragraph)
        story_with_images_desc[f'chapter_{i}'][f'paragraph_{j}'] = dict()
        story_with_images_desc[f'chapter_{i}'][f'paragraph_{j}']['text'] = paragraph
        story_with_images_desc[f'chapter_{i}'][f'paragraph_{j}']['image_desc'] = paragraph_image_desc

story_with_images_desc
        
        


Chapitre 1: Les débuts de Léa
Chapitre 2: La quête de Léa
Chapitre 3: Les rencontres magiques de Léa
Chapitre 4: Le piège diabolique
Chapitre 5: La victoire de Léa


{'chapter_0': {'title': 'Chapitre 1: Les débuts de Léa',
  'paragraph_0': {'text': 'Léa, une jeune sorcière en formation, vivait dans un petit village paisible entouré de forêts enchantées',
   'image_desc': 'Léa, a young witch-in-training, resided in a serene village nestled amidst enchanting forests.'},
  'paragraph_1': {'text': "Depuis toute petite, elle rêvait de devenir une grande sorcière et d'aider les habitants de son village avec ses pouvoirs magiques",
   'image_desc': 'A young girl, surrounded by a quaint village, dreams of becoming a powerful witch to help its inhabitants.'},
  'paragraph_2': {'text': "Un jour, son plus grand rêve se réalisa lorsqu'elle fut acceptée à l'Académie des Sorcières. Elle quitta son village avec enthousiasme, les yeux remplis d'étoiles",
   'image_desc': 'A young girl, with excitement shining in her eyes, joyfully leaves her village as she sets off to the Witch Academy.'},
  'paragraph_3': {'text': 'Mais à peine arrivée, elle fut confrontée à un d

In [43]:
formated_story[0][1][0]

'Léa, une jeune sorcière en formation, vivait dans un petit village paisible entouré de forêts enchantées'

In [59]:
chapter_visual_descriptions = chapter_to_images_desc(formated_story[0][1][1])
chapter_visual_descriptions

'A young girl, surrounded by an enchanting forest, daydreams about becoming a powerful sorceress and using her magical abilities to assist her village.'

'Léa, a young witch-in-training, lived in a tranquil village surrounded by enchanting forests.'

In [ ]:
print(chapter_visual_descriptions)

Paragraph 1: Léa, a young witch-in-training, lived in a peaceful little village surrounded by enchanted forests.
Description: Léa, a young girl with long, flowing hair and a pointed hat, stood in the center of a quaint village. Colorful flowers bloomed along the cobblestone paths, while tall, mystical trees loomed in the background, their branches swaying gently in the breeze.

Paragraph 2: Since she was a little girl, she dreamed of becoming a great witch and helping the villagers with her magical powers.
Description: Léa, now a teenager, sat on a hill overlooking the village, her eyes sparkling with determination. Her colorful robes billowed around her, and a glimmering wand was clasped tightly in her hand. In the distance, the village unfolded like a picturesque tapestry, displaying cozy cottages and smiling faces.

Paragraph 3: One day, her greatest dream came true when she was accepted into the Witch Academy. She left her village with enthusiasm, her eyes filled with stars.
Descri

In [48]:
def image_desc_to_dict(image_desc):
    # regex split into list of dict {chunk: xx, description: xx}
    liste = re.split(r"chunk: (.*)\nvisual description: (.*)", image_desc)
    # remove whitespace
    liste = [x.strip() for x in liste if x.strip()]
    for i in range(0, len(liste), 2):
        liste[i] = {"chunk": liste[i], "description": liste[i+1]}
    return liste

In [49]:
image_desc_to_dict(chapter_visual_descriptions)

[{'chunk': 'Léa était une jeune sorcière en formation.',
  'description': 'A young witch named Léa, with curly black hair and a pointed hat, was surrounded by magical books and potions in a cozy room filled with flickering candlelight.'},
 'A young witch named Léa, with curly black hair and a pointed hat, was surrounded by magical books and potions in a cozy room filled with flickering candlelight.',
 {'chunk': 'Elle vivait dans un petit village entouré de montagnes enchantées.',
  'description': 'The village, nestled at the base of towering enchanted mountains, was adorned with quaint cottages and colorful gardens, where fairies flitted about and unicorns grazed peacefully.'},
 'The village, nestled at the base of towering enchanted mountains, was adorned with quaint cottages and colorful gardens, where fairies flitted about and unicorns grazed peacefully.',
 {'chunk': 'Mais un jour, un sortilège maléfique se répandit sur le village, transformant tous les habitants en statues de pierr

In [9]:
response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
              {"role":"user", "content": f"""Léa était une jeune sorcière en devenir, toujours prête à relever de nouveaux défis. Un matin, elle reçut une missive mystérieuse lui demandant de se rendre au village voisin. Intriguée, Léa fit ses bagages et partit immédiatement. Arrivée sur place, elle découvrit que le village était plongé dans un sommeil profond. Les habitants étaient incapables de se réveiller. Intrépide, Léa savait qu'elle devait agir. Elle se dirigea vers la vieille tour au sommet de la colline, déterminée à découvrir la cause de ce maléfice.
},
              ]
)
response.choices[0].message.content

"chunk: Léa était une jeune sorcière en devenir, toujours prête à relever de nouveaux défis.\nvisual description: \nSomeone, a young witch-to-be, with sparkling eyes and a mischievous smile, stood in front of a bubbling cauldron. Filled with determination, she eagerly prepared for every adventure that came her way.\n\nchunk: Un matin, elle reçut une missive mystérieuse lui demandant de se rendre au village voisin.\nvisual description: \nSomeone, with a curious expression on her face, was standing in her cluttered room. Sunlight peeked through the curtains as she held an old letter, its calligraphic letters whispering secrets that piqued her interest. \n\nchunk: Intriguée, Léa fit ses bagages et partit immédiatement.\nvisual description: \nSomeone, with a determined look in her eyes, swiftly packed her magical potions, spell books, and a few essential belongings. Her cloak billowed behind her as she stepped out of her cottage, ready to embark on a mysterious journey.\n\nchunk: Arrivée s

In [10]:
print(response.choices[0].message.content)

chunk: Léa était une jeune sorcière en devenir, toujours prête à relever de nouveaux défis.
visual description: 
Someone, a young witch-to-be, with sparkling eyes and a mischievous smile, stood in front of a bubbling cauldron. Filled with determination, she eagerly prepared for every adventure that came her way.

chunk: Un matin, elle reçut une missive mystérieuse lui demandant de se rendre au village voisin.
visual description: 
Someone, with a curious expression on her face, was standing in her cluttered room. Sunlight peeked through the curtains as she held an old letter, its calligraphic letters whispering secrets that piqued her interest. 

chunk: Intriguée, Léa fit ses bagages et partit immédiatement.
visual description: 
Someone, with a determined look in her eyes, swiftly packed her magical potions, spell books, and a few essential belongings. Her cloak billowed behind her as she stepped out of her cottage, ready to embark on a mysterious journey.

chunk: Arrivée sur place, ell

In [ ]:
# make a list of dict of chunks and descriptions
